In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class Diputados(scrapy.Spider):
    name = "spider_diputados"
    
    def start_requests(self):
        links = [urlCurr + i for i in terminaciones]
        for link in links:
            yield scrapy.Request( url = link, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'}, 
                              callback = self.parse )
    
    def parse(self,response):
        nombre = response.css('td.textocurrienc > center > strong::text').get()
        idDipu = ''.join([c for c in nombre[1:] if c.isupper()])
        cumple = response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[6]
        curp = creaCurp(nombre,cumple)
        dipus[curp]={}
        dipus.get(curp,{})['nombre']=nombre
        dipus.get(curp,{})['tipoElección']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract_first()
        dipus.get(curp,{})['entidad']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[1]
        dipus.get(curp,{})['distrito']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[2]
        # dc_dict['?????']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[3]
        dipus.get(curp,{})['correo']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[4]
        dipus.get(curp,{})['extensión']=response.css('td.textocurri:nth-of-type(2) > strong::text').extract()[5]
        dipus.get(curp,{})['onomástico']=cumple
        dipus.get(curp,{})['suplente']=response.css('td.textocurri:nth-of-type(2)::text').extract()[6]
        link_asis=urlDipu + response.css('table.tableInfo  a::attr(href)').extract()[0]
        link_votaciones=urlDipu + response.css('table.tableInfo  a::attr(href)').extract()[1]
        yield response.follow(url=link_asis, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse2)
        yield response.follow(url=link_votaciones, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse4)
    
    def parse2(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse3)
    
    def parse3(self,response):
        curp = response.meta['curp']
        prueba = response.xpath('//tr/td/div/font/text()').extract()
        dias = list(filter(lambda a: (a == 'A' or a =='AC'), prueba))
        dipus.get(curp,{})['Asistencias']=dipus.get(curp,{}).get('Asistencias',0) + len(dias)
        #dic1['Asistencias']=dic1.get('Asistencias',0) + len(dias)

    def parse4(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse5)
                
    def parse5(self,response):
        curp = response.meta['curp']
        aux = response.xpath('//td[@class="smallVerde"]/text()').extract()
        aux2={}
        for i in range(1,len(aux),4):
            votos = {}
            votacion = response.xpath('//td[@class="smallVerde"]/text()').extract()[i]
            posi = response.xpath('//td[@class="smallVerde"]/text()').extract()[i+2]
            votos['votación']=votacion
            votos['posición']=posi
            aux2['votos']=aux2.get('votos', []) + [votos]
            #dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos', []) + [votos]
        dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos',0)+len(aux2.get('votos', [])) # Se comenta si la de arriba se descomenta


        

In [3]:
def creaCurp(nombre,cumple):
    
    iniciales = ''.join([c for c in nombre[1:] if c.isupper()])
    c = cumple.replace('-', ' ').split(' ')
    if(len(c[0])==1):
        c[0]="0"+c[0] 
    curp = iniciales[0:3]+c[0]+c[1][0:3]+c[2][-2:]
    return curp

In [4]:
dipus={}
curp = "h"
urlDipu = "http://sitl.diputados.gob.mx/LXIV_leg/"
urlCurr = "http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt="

terminaciones = []
for i in range(0,1):
    for j in range(0,1):
        for k in range(1,3):
            terminaciones.append(str(i)+str(j)+str(k))

# Run the Spider
process = CrawlerProcess()
process.crawl(Diputados)
process.start()


2021-03-13 03:01:00 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-13 03:01:00 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Windows-10-10.0.19041-SP0
2021-03-13 03:01:00 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-13 03:01:00 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-13 03:01:00 [scrapy.extensions.telnet] INFO: Telnet Password: c6185ed47e84a4cb
2021-03-13 03:01:00 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-03-13 03:01:00 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddl

2021-03-13 03:01:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=2&pert=3> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=002)
2021-03-13 03:01:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=2&pert=13> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=002)
2021-03-13 03:01:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_por_pernplxiv.php?iddipt=2&pert=16> (referer: http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=002)
2021-03-13 03:01:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitl.diputados.gob.mx/LXIV_leg/votaciones_diputados_xperiodonplxiv.php?dipt=001> (referer: http://sitl.diputados.gob.mx/LXIV_leg/curricula.php?dipt=001)
2021-03-13 03:01:06 [scr

2021-03-13 03:01:13 [scrapy.core.engine] INFO: Spider closed (finished)


In [5]:
print(dipus)

{'ELT15mar77': {'nombre': 'Dip. Elba Lorena Torres Díaz (LICENCIA)', 'tipoElección': 'Mayoría Relativa', 'entidad': 'Aguascalientes ', 'distrito': '2 ', 'correo': 'elba.torres@diputados.gob.mx ', 'extensión': '62100 ', 'onomástico': '15-marzo-1977 ', 'suplente': 'Haydeé Hernández Muñoz\n\t\t\t\t\t\n\n\n ', 'Asistencias': 130, 'votos': 585}, 'FJL01jul79': {'nombre': 'Dip. Francisco Javier Luévano Núñez', 'tipoElección': 'Mayoría Relativa', 'entidad': 'Aguascalientes ', 'distrito': '1 ', 'correo': 'francisco.luevano@diputados.gob.mx ', 'extensión': '59907 ', 'onomástico': '1-julio-1979 ', 'suplente': 'Gustavo Armendariz Viramontes\n\t\t\t\t\t\n\n\n ', 'Asistencias': 140, 'votos': 589}}
